# 枠回しのアルゴリズム

In [14]:
import cvxpy
import numpy as np
import pandas as pd
import time

[cvxpyの情報](https://www.cvxpy.org/tutorial/advanced/index.html)
<b></b>

```
Variable()     # スカラーを表す。
Variable(5)    # ベクトルを表す。
Variable(4, 7) # 行列を表す。
```

## ○線型計画法を用いる。
__本当は、［SB：M1834 花王：主婦］が指定階層であるが、<br>
データの関係上、<font color='Red'>［SB：女30~49 花王：女18~34］</font>とする。<br>
また、<font color='Red'>号数は等しく4（既にあるデータ）</font>とする。__

In [13]:
# データの読み込み。
df_SB = pd.read_excel('input/SB予測_201808.xlsx').loc[:, ['世帯','女 18～34才','女 30～39才']]
df_kao = pd.read_excel('input/花王予測_201808.xlsx').loc[:, ['世帯','女 18～34才','主婦']].rename(columns={'主婦':'女 30～39才'})

In [26]:
SB_target  = '女 30～39才'
kao_target = '女 18～34才'
SB_gou  = '世帯'
kao_gou = '世帯'

In [22]:
# 同じ向きに引くことがポイント！！
df_SB['value'] = df_SB[SB_target] - df_SB[kao_target]
df_kao['value'] = df_kao[SB_target] - df_kao[kao_target]

In [30]:
print("データ数:", len(df_SB))
df_SB.head(3)

データ数: 127


,世帯,女 18～34才,女 30～39才,value
1,6.1,3.5,4.6,1.1
2,9.0,2.6,4.1,1.5
3,10.1,2.1,3.0,0.9


In [29]:
print("データ数:", len(df_kao))
df_kao.head(3)

データ数: 2268


,世帯,女 18～34才,女 30～39才,value
1,8.7,4.0,7.6,3.6
2,5.1,2.3,4.1,1.8
3,5.1,2.3,4.1,1.8


#### 相対価値のところで逆向きに引いた方（花王）の値を全てひっくり返す！

In [65]:
df_kao = df_kao.applymap(lambda x:-x)
df_kao.head(3)

,世帯,女 18～34才,女 30～39才,value
1,-8.7,-4.0,-7.6,-3.6
2,-5.1,-2.3,-4.1,-1.8
3,-5.1,-2.3,-4.1,-1.8


In [32]:
# そのままくっつける。
df = pd.concat([df_SB, df_kao]).reset_index(drop=True)

In [56]:
df.head(3)

,世帯,女 18～34才,女 30～39才,value
0,6.1,3.5,4.6,1.1
1,9.0,2.6,4.1,1.5
2,10.1,2.1,3.0,0.9


In [54]:
# 小数第一位までにする。
df = df.round(1)

***
***

In [62]:
start = time.time()

Pai = df[SB_gou].values # SBの指定号数に従った枠の視聴率
Pbi = df[kao_gou].values # 花王の指定号数に従った枠の視聴率
Wi = df['value'].values # 視聴率の上昇量(これを最大化したい)
Ci = cvxpy.Variable(len(Pai), boolean=True) # 取得するかどうかのフラグ
objective = cvxpy.Maximize(Wi * Ci) # 最大化したいもの（＝視聴率の上昇量）
constraints = [Pai * Ci == 0] + [Pbi * Ci == 0] # SB,kaoの号数が変わらないという制約条件
prob = cvxpy.Problem(objective, constraints) # 解く問題のこと。
prob.solve(solver=cvxpy.ECOS_BB) # solverを変える事で、解き方が変わる。
result = [round(ix, 0) for ix in Ci.value] # これで、フラグが0, 1で表される。

end = time.time()
print("データ数",len(df),"で処理時間は", end-start, "秒")

データ数 2395 で処理時間は 44.23177719116211 秒


In [ ]:
print("取得する枠かのフラグ:", result)

In [4]:
print("取得する枠の視聴率の上昇値:", np.array(result)*Wi)

取得する枠の視聴率の上昇値: [22.  0. 16. 10. 35.  0. 42. 53.]


In [5]:
print("上昇値の合計値:", sum(np.array(result)*Wi))

上昇値の合計値: 178.0


***
***

## 実際に算出された号数を用いて実践する

In [2]:
# データの読み込み。
df_CCJC = pd.read_excel('sample/df_CCJC_sample.xlsx').loc[:, ["6_gou","世帯","女 20～34才","主婦"]].reset_index(drop=True)
df_kao = pd.read_excel('sample/df_kao_sample.xlsx').loc[:, ["6_gou","世帯","女 18～34才","主婦"]].rename(columns={"女 18～34才":"女 20～34才"}).reset_index(drop=True)

In [3]:
# 同じ向きに引くことがポイント！！
df_CCJC['value'] = df_CCJC["女 20～34才"]-df_CCJC["主婦"]
df_kao['value'] = df_kao["女 20～34才"]-df_kao["主婦"]

In [5]:
print("データ数:", len(df_kao))
df_kao.head(3)

データ数: 50


,id,6_gou,世帯,女 20～34才,主婦,value
0,1,10.0,10.4,5.7,8.1,-2.4
1,2,6.6,6.1,4.1,5.4,-1.3
2,3,6.6,6.1,4.1,5.4,-1.3


In [6]:
print("データ数:", len(df_CCJC))
df_CCJC.head(3)

データ数: 50


,id,6_gou,世帯,女 20～34才,主婦,value
0,1,6.6,8.7,3.9,6.5,-2.6
1,2,5.5,5.7,3.5,4.2,-0.7
2,3,5.5,5.7,3.5,4.2,-0.7


In [7]:
df_kao = df_kao.applymap(lambda x:-x)
df_kao.head(3)

,id,6_gou,世帯,女 20～34才,主婦,value
0,-1,-10.0,-10.4,-5.7,-8.1,2.4
1,-2,-6.6,-6.1,-4.1,-5.4,1.3
2,-3,-6.6,-6.1,-4.1,-5.4,1.3


In [8]:
# そのままくっつける。
df = pd.concat([df_kao, df_CCJC]).reset_index(drop=True)

In [9]:
# 小数第一位までにする。
df = df.round(1)

In [15]:
start = time.time()

Pai = df['6_gou'].values # 花王の指定号数に従った枠の視聴率
Pbi = df['世帯'].values # CCJCの指定号数に従った枠の視聴率
Wi = df['value'].values # 視聴率の上昇量(これを最大化したい)
Ci = cvxpy.Variable(len(Pai), boolean=True) # 取得するかどうかのフラグ
objective = cvxpy.Maximize(Wi * Ci) # 最大化したいもの（＝視聴率の上昇量）
constraints = [Pai * Ci == 0] + [Pbi * Ci == 0] # A,Bの号数が変わらないという制約条件
prob = cvxpy.Problem(objective, constraints) # 解く問題のこと。
prob.solve(solver=cvxpy.ECOS_BB) # solverを変える事で、解き方が変わる。
result = [round(ix, 0) for ix in Ci.value] # これで、フラグが0, 1で表される。

end = time.time()
print("データ数",len(df),"で処理時間は", end-start, "秒")

データ数 100 で処理時間は 1.4606945514678955 秒


In [ ]:
print("取得する枠かのフラグ:", result)

In [4]:
print("取得する枠の視聴率の上昇値:", np.array(result)*Wi)

取得する枠の視聴率の上昇値: [22.  0. 16. 10. 35.  0. 42. 53.]


In [5]:
print("上昇値の合計値:", sum(np.array(result)*Wi))

上昇値の合計値: 178.0
